# December 03, 2021

https://adventofcode.com/2021/day/3

In [1]:
import pandas as pd
import numpy as np

# Part 1

In [2]:
# read as string to get leading 0s
df = pd.read_csv("data/2021/03.txt", dtype=str, names=["diag"])

In [ ]:
# Parse individual digits
dignames = [str(x) for x in range( len(df["diag"][0]) )[::-1] ]
df[dignames] = df.apply(lambda x: [int(d) for d in x["diag"]], axis=1, result_type="expand")

In [ ]:
# We could take a shortcut here using the fact that it's binary, but let's be more general
# get digit counts
digc = df[dignames].apply( lambda x: x.value_counts() )

In [ ]:
gamma = pd.DataFrame(digc.apply(lambda x: x.idxmax(), axis=0), columns=["digit"])
gamma["pow"] = gamma.index
gamma["val"] = gamma.apply( lambda x: 2**int(x["pow"]) * x["digit"], axis=1)
gamma

In [ ]:
epsilon = pd.DataFrame(digc.apply(lambda x: x.idxmin(), axis=0), columns=["digit"])
epsilon["pow"] = epsilon.index
epsilon["val"] = epsilon.apply( lambda x: 2**int(x["pow"]) * x["digit"], axis=1 )
epsilon

In [ ]:
gam = gamma["val"].sum()
eps = epsilon["val"].sum()
print(gam, eps, gam*eps)

# Part 2

In [ ]:
df = df.iloc[:11]

In [ ]:
def select_max(x):
    vc = x.value_counts()
    # only one value present, return it
    if len(vc) == 1:
        return vc.index[0]
    
    if vc[0] > vc[1]:
        return 0
    else:
        return 1
    
def select_min(x):
    vc = x.value_counts()
    # if only one value present, return the other
    if len(vc) == 1:
        return 1 - vc.index[0]
    
    if vc[0] <= vc[1]:
        return 0
    else:
        return 1
    
def select_bit(x, which):
    if which == "min":
        return select_min(x)
    else:
        return select_max(x)
    
def filter_values( df, pos, which ):
    targ = select_bit( df[pos], which )

    return df.loc[ df[pos] == targ ]

def get_rating( df, which, debug = 0 ):
    cols = [x for x in df.columns if x != "diag"]
    cidx = 0

    rcnt = df.shape[0]
    while rcnt > 1 and cidx < len(cols):
        c = cols[cidx]
        print("Checking col", c)
        df = filter_values( df, cols[cidx], which )
        rcnt = df.shape[-]
        if debug >= 1:
            print("rows left:", rcnt)
        if debug == 2:
            display(df)

        cidx += 1
    assert rcnt > 0, "All values eliminated!"
    assert rcnt == 1, "Multiple values remaining!"

    return df

def read_val( df ):
    cols = [x for x in df.columns if x != "diag"]
    vals = [ 2**int(c) * df[c].iloc[0] for c in cols ]
    return sum(vals)

In [ ]:
o2 = get_rating(df, "max", debug=0)
print(o2, "\n----------\n")
co2 = get_rating(df, "min", debug=0)
print(co2)

In [ ]:
o2_rating = read_val(o2)
co2_rating = read_val(co2)
print( o2_rating, co2_rating, o2_rating*co2_rating )

In [ ]:
display(o2)
display(co2)

In [ ]:
512+128+64+4+1